# Item Metadata

In [ ]:
# TODO move to nondirectional

In [ ]:
using DataFrames

import CSV
import Statistics: mean
import StatsBase

In [ ]:
# import NBInclude: @nbinclude
# @nbinclude("../Alpha.ipynb");

In [ ]:
# function fill_feature(split, implicit, transpose, user_feature)
#     users = get_split(split, implicit; transpose = transpose).user
#     feature = zeros(length(users))
#     @tprogress Threads.@threads for j = 1:length(users)
#         if users[j] <= length(user_feature)
#             feature[j] = user_feature[users[j]]
#         end
#     end
#     feature
# end;

In [ ]:
# function count_feature(split, implicit, transpose)
#     # when transpose is true, returns the number of rated items per user
#     # when transpose is false, return the number of rated users per item
#     users = get_split("training", implicit; transpose = transpose).user
#     user_to_num_training_items = zeros(maximum(users), Threads.nthreads())
#     @tprogress Threads.@threads for i = 1:length(users)
#         user_to_num_training_items[users[i], Threads.threadid()] += 1
#     end
#     user_to_num_training_items = sum(user_to_num_training_items, dims = 2)
#     fill_feature(split, implicit, transpose, user_to_num_training_items)
# end;

In [ ]:
# function bias_feature(split, implicit, transpose)
#     bias_param = transpose ? "a" : "u"
#     users = get_split("training", implicit; transpose = transpose).user
#     user_bias = read_params("ExplicitUserItemBiases")[bias_param]
#     fill_feature(split, implicit, transpose, user_bias)
# end;

In [ ]:
# function std_feature(split, implicit, transpose)
#     # get sum squared error
#     users = get_split("training", implicit; transpose = transpose).user
#     ratings = get_split("training", implicit; transpose = transpose).rating
#     params = transpose ? ["a", "u"] : ["u", "a"]
#     user_means =
#         read_params("ExplicitUserItemBiases")[params[1]] .+
#         mean(read_params("ExplicitUserItemBiases")[params[2]])
#     user_to_training_std = zeros(maximum(users), Threads.nthreads())
#     user_to_num_training_items = zeros(maximum(users), Threads.nthreads())
#     @tprogress Threads.@threads for i = 1:length(users)
#         u = users[i]
#         user_to_training_std[u, Threads.threadid()] += (ratings[u] - user_means[u])^2
#         user_to_num_training_items[u, Threads.threadid()] += 1
#     end
#     user_to_training_std = sum(user_to_training_std, dims = 2)
#     user_to_num_training_items = sum(user_to_num_training_items, dims = 2)

#     # transform to std
#     @tprogress Threads.@threads for u = 1:maximum(users)
#         if user_to_num_training_items[u] > 0
#             user_to_training_std[u] /= user_to_num_training_items[u]
#         end
#     end
#     user_to_training_std .= sqrt.(user_to_training_std)
#     fill_feature(split, implicit, transpose, user_to_training_std)
# end;

In [ ]:
# function popularity_feature(split, implicit, transpose)
#     # returns the average popularity of all items the user has seen
#     users = get_split("training", implicit; transpose).user
#     items = get_split("training", implicit; transpose).item
#     item_means = read_params("ExplicitUserItemBiases")[transpose ? "u" : "a"]
#     user_to_avg_item_pop = zeros(maximum(users), Threads.nthreads())
#     user_to_num_training_items = zeros(maximum(users), Threads.nthreads())
#     @tprogress Threads.@threads for i = 1:length(users)
#         u = users[i]
#         a = items[i]
#         user_to_avg_item_pop[u, Threads.threadid()] += item_means[a]
#         user_to_num_training_items[u, Threads.threadid()] += 1
#     end
#     user_to_avg_item_pop = sum(user_to_avg_item_pop, dims = 2)
#     user_to_num_training_items = sum(user_to_num_training_items, dims = 2)

#     @tprogress Threads.@threads for u = 1:maximum(users)
#         if user_to_num_training_items[u] > 0
#             user_to_avg_item_pop[u] /= user_to_num_training_items[u]
#         end
#     end
#     fill_feature(split, implicit, transpose, user_to_avg_item_pop)
# end;

In [ ]:
function date_to_year(x)
    if ismissing(x)
        return NaN
    end
    Dates.year(x) + Dates.month(x) / 12
end

function get_anime()
    anime_to_uid = DataFrame(CSV.File(get_data_path("processed_data/anime_to_uid.csv")))
    anime_to_uid.uid .+= 1
    anime = DataFrame(CSV.File(get_data_path("processed_data/anime.csv"), ntasks = 1))
    anime = innerjoin(anime_to_uid, anime, on = "anime_id")
    anime.start_year = date_to_year.(anime[:, "start_date"])
    anime.end_year = date_to_year.(anime[:, "end_date"])
    anime
end;

In [ ]:
using NBInclude
@nbinclude("../Alpha.ipynb")

In [ ]:
function get_anime_column(anime_to_uid, col)
    #TODO is there a better way of handling missing values?
    function is_missing(x)
        if ismissing(x)
            return true
        end
        if isa(x, Real)
            return isnan(x)
        end
        return x == "NaN"
    end
    default = StatsBase.mode(filter(x -> !is_missing(x), anime_to_uid[:, col]))
    anime_to_col = fill(default, num_items())
    for i = 1:size(anime_to_uid)[1]
        val = anime_to_uid[i, col]
        if !is_missing(val)
            anime_to_col[anime_to_uid[i, "uid"]] = val
        end
    end
    anime_to_col
end;

In [ ]:
function item_feature(col, out_users, out_items; categorical)
    # subset column of interest
    anime_to_uid = get_anime()
    anime_to_col = get_anime_column(anime_to_uid, col)

    # do a 1-hot encoding for categorical variables
    encoding_fn = x -> anime_to_col[x]
    ncols = 1
    if categorical
        encoding = sort(collect(Set(collect(anime_to_col))))
        @debug "$col categories: $encoding"
        function one_hot_encoding(item)
            if item > length(anime_to_col)
                return zeros(Float32, length(encoding))
            end
            encoding .== anime_to_col[item]
        end
        encoding_fn = one_hot_encoding
        ncols = length(encoding)
    end

    feature = zeros(Float32, length(out_items), ncols)
    @tprogress Threads.@threads for j = 1:length(out_items)
        feature[j, :] .= encoding_fn(out_items[j])
    end
    feature
end;

In [ ]:
# number of items of the same category that the user has seen
function item_count_feature(col, training, out_users, out_items, to_category = identity;)
    # subset column of interest
    anime_to_uid = get_anime()
    anime_to_uid[!, col] = to_category.(anime_to_uid[:, col])
    anime_to_col = get_anime_column(anime_to_uid, col)

    categories = sort(collect(Set(collect(anime_to_col))))
    @debug "$col categories: $categories"
    encoding(item) = findfirst(x -> x == anime_to_col[item], categories)

    # get the number of items per user-category
    users = training.user
    items = training.item
    user_to_num_training_items =
        zeros(Float32, maximum(users), length(categories), Threads.nthreads())
    @tprogress Threads.@threads for i = 1:length(users)
        user_to_num_training_items[users[i], encoding(items[i]), Threads.threadid()] += 1
    end
    user_to_num_training_items = sum(user_to_num_training_items, dims = 3)

    feature = zeros(Float32, length(out_items))
    @tprogress Threads.@threads for i = 1:length(out_items)
        feature[i] = user_to_num_training_items[out_users[i], encoding(out_items[i])]
    end
    feature
end;

In [ ]:
function parse_string_array(x)
    # handle nested quotes
    x = replace(x, "' " => "", "'s " => "s ", "\"" => "'")
    # parse as a julia array
    eval(Meta.parse(replace(x, "'" => '"')))
end

function parse_string_dict(x)
    # handle nested quotes
    x = replace(
        x,
        "' " => "",
        "'s " => "s ",
        "\"" => "'",
        "{" => "Dict(",
        "}" => ")",
        ":" => "=>",
        "M's" => "Ms",
        "6'o" => "6o",
    )    # parse as a julia array
    eval(Meta.parse(replace(x, "'" => '"')))
end

function embed(tags::Vector, tag_embedding::Dict)
    x = zeros(length(tag_embedding))
    for i = 1:length(tags)
        x[tag_embedding[tags[i]]] = 1
    end
    x
end

function get_tags(col)
    df = get_anime()
    tags = parse_string_array.(df[:, col])
    tag_embedding = sort(collect(Set(reduce(vcat, tags))))
    tag_embedding = Dict(tag_embedding[i] => i for i = 1:length(tag_embedding))
    X = zeros(length(tag_embedding), num_items())
    for i = 1:length(df.uid)
        X[:, df.uid[i]] = embed(tags[i], tag_embedding)
    end
    X
end

function get_ids(x::Vector)
    ids = []
    for y in x
        push!(ids, y["id"])
    end
    ids
end

function get_ids(col::String)
    df = get_anime()
    ids = get_ids.(parse_string_dict.(df[:, col]))
    id_embedding = sort(collect(Set(reduce(vcat, ids))))
    id_embedding = Dict(id_embedding[i] => i for i = 1:length(id_embedding))
    X = zeros(length(id_embedding), num_items())
    for i = 1:length(df.uid)
        X[:, df.uid[i]] = embed(ids[i], id_embedding)
    end
    X
end;

In [ ]:
function normalize!(X)
    # put X in the range [-1, 1]
    for i = 1:size(X)[1]
        x = X[i, :]
        X[i, :] .= (x .- minimum(x)) ./ (maximum(x) - minimum(x) .+ eps(Float32))
    end
    2 .* X .- 1
end

function standardize!(X)
    # gaussianize X
    for i = 1:size(X)[1]
        x = X[i, :]
        X[i, :] .= (x .- mean(x)) ./ (std(x) .+ eps(Float32))
    end
    X
end

function get_neural_item_features()
    M1 = collect(
        hcat(
            item_feature(
                "start_year",
                fill(1, num_items()),
                1:num_items(),
                categorical = false,
            ),
            item_feature(
                "end_year",
                fill(1, num_items()),
                1:num_items(),
                categorical = false,
            ),
            log.(
                1 .+ item_feature(
                    "num_episodes",
                    fill(1, num_items()),
                    1:num_items(),
                    categorical = false,
                ),
            ),
            log.(
                1 .+ item_feature(
                    "average_episode_duration",
                    fill(1, num_items()),
                    1:num_items(),
                    categorical = false,
                ),
            ),
        )',
    )
    M2 = collect(
        hcat(
            item_feature("nsfw", fill(1, num_items()), 1:num_items(), categorical = true),
            item_feature("medium", fill(1, num_items()), 1:num_items(), categorical = true),
            item_feature("source", fill(1, num_items()), 1:num_items(), categorical = true),
        )',
    )
    M3 = get_tags("genres")
    M4 = get_tags("tags")
    M5 = get_ids("studios")
    # scale non-categorical features
    normalize!(M1)
    (vcat(M1, M2), M3, M4, M5)
end;

In [ ]:
function get_item_metadata_features(users::Vector, items::Vector, training::RatingsDataset)
    round_to_multiple(x, n) = round(x / n) * n
    function bucket_num_epiodes(x)
        if isnan(x)
            return x
        end
        if x <= 6
            return round_to_multiple(x, 1)
        end
        if x <= 26
            return round_to_multiple(x, 13)
        end
        if x <= 52
            return round_to_multiple(x, 26)
        end
        return 100
    end
    hcat(
        # # shared user/item features
        # count_feature(split, implicit, true),
        # count_feature(split, implicit, false),
        # bias_feature(split, implicit, true),
        # bias_feature(split, implicit, false),
        # std_feature(split, implicit, true),
        # std_feature(split, implicit, false),
        # popularity_feature(split, implicit, true),
        # popularity_feature(split, implicit, false),

        # item only features
        item_feature("start_year", users, items, categorical = false),
        item_feature("end_year", users, items, categorical = false),
        item_feature("nsfw", users, items, categorical = true),
        item_feature("medium", users, items, categorical = true),
        item_feature("num_episodes", users, items, categorical = false),
        item_feature("source", users, items, categorical = true),
        item_feature("average_episode_duration", users, items, categorical = false),

        # number of series the user has seen in each category
        item_count_feature("start_year", training, users, items, x -> round_to_multiple(x, 5)),
        item_count_feature("end_year", training, users, items, x -> round_to_multiple(x, 5)),
        item_count_feature("nsfw", training, users, items),
        item_count_feature("medium", training, users, items),
        item_count_feature("num_episodes", training, users, items, bucket_num_epiodes),
        item_count_feature("source", training, users, items),
        item_count_feature(
            "average_episode_duration",
            training,
            users,
            items,
            x -> x <= 1800 ? round_to_multiple(x, 600) : round_to_multiple(x, 1800),
        ),
    )
end;

In [ ]:
function get_item_metadata_features(split::String, implicit::Bool)
    df = get_split(split, implicit)
    training = get_split("training", false)    
    get_item_metadata_features(df.user, df.item, training)
end